In [1]:
# Import necessary libraries
import pandas as pd
import yfinance as yf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib

# Function to compute RSI (Relative Strength Index)
def compute_rsi(series, period=14):
    delta = series.diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Feature Engineering
def add_technical_indicators(df):
    df['MA10'] = df['Close'].rolling(window=10).mean()
    df['MA100'] = df['Close'].rolling(window=100).mean()
    df['EMA10'] = df['Close'].ewm(span=10, adjust=False).mean()
    df['RSI'] = compute_rsi(df['Close'])
    df['Volume'] = df['Volume']
    df.dropna(inplace=True)
    return df

# Load stock data
stock_data = yf.download("TATAMOTORS.NS", start="2000-01-01", end="2024-01-01")

# Add technical indicators
stock_data = add_technical_indicators(stock_data)

# Prepare the data for training
X = stock_data[['MA10', 'MA100', 'EMA10', 'RSI', 'Volume']]
y = stock_data['Close']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Train the Linear Regression model
lr = LinearRegression()
lr.fit(X_train, y_train)

# Evaluate the model
y_pred = lr.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Linear Regression - MSE: {mse}, MAE: {mae}, R2: {r2}")

# Save the model
#Users\sangi\OneDrive\Desktop\seminar2
joblib.dump(lr, 'C:/Users/sangi/OneDrive/Desktop/seminar2/mymodels/linear_regression_model.pkl')
print("Linear Regression model saved as linear_regression_model.pkl")

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

Linear Regression - MSE: 56.0053962702879, MAE: 4.872329522053011, R2: 0.9981613966081953
Linear Regression model saved as linear_regression_model.pkl


In [3]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# Function to create LSTM model
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=100, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.3))
    model.add(LSTM(units=100, return_sequences=False))
    model.add(Dropout(0.3))
    model.add(Dense(units=50))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model
# Load stock data
stock_data = yf.download("TATAMOTORS.NS", start="2000-01-01", end="2024-12-01")
# Prepare the data for LSTM
data = stock_data[['Close']].values
print(stock_data.head())  # Check if data is fetched
print(stock_data.shape)  # Ensure it has rows
scaler = MinMaxScaler(feature_range=(0, 1))  # Normalize the data
scaled_data = scaler.fit_transform(data)
# Define training data length (80% for training)
training_data_len = int(np.ceil(0.8 * len(scaled_data)))
# Split the data into training and testing sets
train_data = scaled_data[0:training_data_len, :]
test_data = scaled_data[training_data_len - 120:, :]
# Create sequences
def create_sequences(data, seq_length=120):
    X = []
    y = []
    for i in range(seq_length, len(data)):
        X.append(data[i-seq_length:i, 0])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

X_train, y_train = create_sequences(train_data)    # X_train: [[120 days of prices], [120 days of prices], ...]
                                                   # y_train: [Price on day 121, Price on day 122, ...]

X_test, y_test = create_sequences(test_data)
# Reshape for LSTM input
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
# Create the LSTM model
lstm_model = create_lstm_model((X_train.shape[1], 1))
# Train the model 
lstm_model.fit(X_train, y_train, epochs=50, batch_size=64, verbose=1)
# Evaluate the model
predictions = lstm_model.predict(X_test)
predictions = scaler.inverse_transform(predictions)  # Convert back to original scale
y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))
mse = mean_squared_error(y_test_actual, predictions)
mae = mean_absolute_error(y_test_actual, predictions)
r2 = r2_score(y_test_actual, predictions)
print(f"LSTM - MSE: {mse}, MAE: {mae}, R2: {r2}")
# Save the LSTM model
lstm_model.save('C:/Users/sangi/OneDrive/Desktop/seminar2/mymodels/lstm_model.h5')
print("LSTM model saved as lstm_model.h5")


[*********************100%***********************]  1 of 1 completed
C:\Users\sangi\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Price              Close          High           Low          Open  \
Ticker     TATAMOTORS.NS TATAMOTORS.NS TATAMOTORS.NS TATAMOTORS.NS   
Date                                                                 
2000-01-03     32.728165     32.803664     31.316363     31.316363   
2000-01-04     31.437164     33.067911     31.104974     32.765919   
2000-01-05     32.199680     32.886706     29.293024     29.293024   
2000-01-06     33.535992     34.713751     32.463925     32.463925   
2000-01-07     36.223701     36.223701     33.686985     33.822882   

Price             Volume  
Ticker     TATAMOTORS.NS  
Date                      
2000-01-03       3528277  
2000-01-04       3544397  
2000-01-05       5849540  
2000-01-06      10274966  
2000-01-07      11477451  
(6218, 5)
Epoch 1/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 19s 173ms/step - loss: 0.0041
Epoch 2/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 13s 167ms/step - loss: 3.9789e-04
Epoch 3/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 12s 159ms/step - loss: 3.3193e-04
E

LSTM - MSE: 327.3846153031263, MAE: 13.33880930108019, R2: 0.9957743789528583
LSTM model saved as lstm_model.h5
